# Creación de función que convierte el iEEG en esprectrograma y label continuo

In [51]:
import torch
import librosa

import numpy                 as np
import pandas                as pd
import utilit_espectrograms  as ue
import torchaudio.transforms as T

from scipy import fft        as sp_fft

In [52]:
root_dir       = '/media/martin/Disco2/Rns_Data/PITT_PI_EEG/'
meta_data_file = '/media/martin/Disco2/Rns_Data/PITT_PI_EEG/METADATA/allfiles_metadata.csv'

df_meta        = pd.read_csv(meta_data_file)
patients       = df_meta['rns_id'].unique().tolist()
s              = 0

train_df       = df_meta.copy()
train_df.drop(train_df[train_df['rns_id'] == patients[s]].index, inplace = True)

In [53]:
# nombre de los datos

iEEG_arr = np.asarray(train_df.iloc[:, 1])

In [55]:
idx = 11

# Get image name from the pandas df
file_name = root_dir + iEEG_arr[idx] + '.npy'

dic   = np.load(file_name, allow_pickle=True)
data  = dic.item().get('iEEG')
label = dic.item().get('label_time')

In [56]:
print('iEEG:',data.shape,'\n')
print('label:',label.shape)

iEEG: torch.Size([4, 22500]) 

label: (2,)


## Transformación a espectrograma

In [57]:
def get_spectrogram_2(signal, fs, n_fft = 256, win_len = None, hop_len = None, power = 2.0):

    wind_dic={'periodic': True, 'beta': 10}

    spectrogram = T.Spectrogram(
                                n_fft=n_fft, 
                                win_length=win_len,
                                hop_length=hop_len, 
                                pad=0,
                                window_fn =torch.kaiser_window,
                                normalized=False,
                                power=power, 
                                wkwargs=wind_dic
                               )

    time   = np.arange(win_len/2, signal.shape[-1] + win_len/2 + 1, win_len - (win_len-hop_len))/float(fs)
    time  -= (win_len/2) / float(fs)

    freqs  = sp_fft.rfftfreq(n_fft, 1/fs)

    spec   = spectrogram(signal)

    # spec to DB
    spec   = librosa.power_to_db(spec, top_db=top_db)

    # save up to 60 Hz
    idx_60 = np.where(freqs <= 60)[0][-1]
    spec   = spec[:, :idx_60,:]

    return spec, time, freqs

In [58]:
# Definición de variables para crear el espectrograma 

ECOG_SAMPLE_RATE = 250
ECOG_CHANNELS    = 4
TT               = 1000 # window length
SPEC_WIN_LEN     = int(ECOG_SAMPLE_RATE * TT / 1000 ) # win size
overlap          = 500 
SPEC_HOP_LEN     = int(ECOG_SAMPLE_RATE * (TT - overlap) / 1000) # Length of hop between windows.
SPEC_NFFT        = 500  # to see changes in 0.5 reso
top_db           = 40.0

spec, t, f = get_spectrogram_2(data, ECOG_SAMPLE_RATE, SPEC_NFFT, SPEC_WIN_LEN, SPEC_HOP_LEN)

## label continuo

In [59]:
time  = np.linspace(0, 90, 181)

Para algun otra prueba estaria bueno pasar el dato del tiempo como punto flotante y no como un entero para que pueda tomar los valores .5. Lo pase de esta forma para no perder informacion

In [60]:
label_time = np.zeros(181)

if label[0] == 1:
    idx_t = np.where(time <= label[1])[0][-1]
    label_time[idx_t]=1

In [63]:
class smoothing_label(object):
    def __call__(self, label, n=5, std=2.5):
        """"apply label smoothing."""
        label_time_smooth = label.copy()
        # for idx in range(len(label)):
        leng  = len(label)
        idx_t = np.where(label==1)[0]
        
        if len(idx_t)!=0:
            if leng  - idx_t < n:
                n = leng  - idx_t
                aux = np.arange(idx_t-n,idx_t+n,1)
            elif idx_t - n < 0 : # in the very begining
                n = n + (idx_t - n)
                aux = np.arange(idx_t-n,idx_t+n,1)
            else:
                aux = np.arange(idx_t-n,idx_t+n,1)
                
            if aux.size!=0:                
                gaus =np.exp(-np.power(aux - idx_t, 2.) / (2 * np.power(std, 2.)))
                label_time_smooth[aux] = gaus
            else:
                label_time_smooth[idx_t] = 1
        return label_time_smooth

In [64]:
target_transform = smoothing_label()

In [65]:
label = target_transform(label_time)

In [66]:
label

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])